In [29]:
# read the data
from pathlib import Path

# Create a Path object for the current directory
current_directory = Path.cwd()
print("Current Directory:", current_directory.resolve())

# Creating a Path object for an example file that does not yet exist
example_file_path = current_directory / "database.csv"

# Reading the contents of the file to test if we have well acess to our data
if example_file_path.exists():
    with example_file_path.open("r") as file:
        first_line = file.readline()
        print(first_line)
else:
    print("The file does not exist.")

Current Directory: C:\Users\33783\Documents\EPFL\Etudes\BA4\Amazing-Project
AAM,ea



In [ ]:
# Request of the user

In [ ]:
# print the yield

In [ ]:
# print the molecule
from rdkit import Chem
from rdkit.Chem import Draw

original_smiles="[Br-:1].[Br:2][C:5]([C:4]([H:3])([C:31]#[N:32])[C:41]#[N:42])([H:11])[H:21]"
# Parse the original SMILES notation
mol = Chem.MolFromSmiles(original_smiles)

# Convert the parsed molecule back to SMILES without atom mapping numbers
smiles_without_numbers = Chem.MolToSmiles(mol, allBondsExplicit=True, isomericSmiles=True)


# Create a molecule object from the SMILES notation
mol = Chem.MolFromSmiles(smiles_without_numbers)

# Draw and display the molecule
img = Draw.MolToImage(mol)

# Display the image
img.show()